In [1]:
!wget https://www.inf.ufpr.br/bakrinski/sports.zip
!unzip sports.zip

A saída de streaming foi truncada nas últimas 5000 linhas.
  inflating: sports/train/gaga/013.jpg  
  inflating: sports/train/gaga/053.jpg  
  inflating: sports/train/gaga/002.jpg  
  inflating: sports/train/gaga/021.jpg  
  inflating: sports/train/gaga/039.jpg  
  inflating: sports/train/gaga/034.jpg  
  inflating: sports/train/gaga/012.jpg  
  inflating: sports/train/gaga/090.jpg  
  inflating: sports/train/gaga/105.jpg  
  inflating: sports/train/gaga/095.jpg  
  inflating: sports/train/gaga/051.jpg  
  inflating: sports/train/gaga/070.jpg  
  inflating: sports/train/gaga/043.jpg  
  inflating: sports/train/gaga/044.jpg  
  inflating: sports/train/gaga/059.jpg  
  inflating: sports/train/gaga/116.jpg  
  inflating: sports/train/gaga/082.jpg  
  inflating: sports/train/gaga/079.jpg  
  inflating: sports/train/gaga/045.jpg  
  inflating: sports/train/gaga/024.jpg  
  inflating: sports/train/gaga/102.jpg  
  inflating: sports/train/gaga/080.jpg  
  inflating: sports/train/gaga/081.jpg 

In [33]:
!wget https://www.inf.ufpr.br/bakrinski/butterflies.zip
!unzip butterflies.zip

A saída de streaming foi truncada nas últimas 5000 linhas.
  inflating: butterflies/train/CAIRNS BIRDWING/090.jpg  
  inflating: butterflies/train/CAIRNS BIRDWING/105.jpg  
  inflating: butterflies/train/CAIRNS BIRDWING/095.jpg  
  inflating: butterflies/train/CAIRNS BIRDWING/051.jpg  
  inflating: butterflies/train/CAIRNS BIRDWING/070.jpg  
  inflating: butterflies/train/CAIRNS BIRDWING/043.jpg  
  inflating: butterflies/train/CAIRNS BIRDWING/044.jpg  
  inflating: butterflies/train/CAIRNS BIRDWING/059.jpg  
  inflating: butterflies/train/CAIRNS BIRDWING/116.jpg  
  inflating: butterflies/train/CAIRNS BIRDWING/082.jpg  
  inflating: butterflies/train/CAIRNS BIRDWING/079.jpg  
  inflating: butterflies/train/CAIRNS BIRDWING/045.jpg  
  inflating: butterflies/train/CAIRNS BIRDWING/024.jpg  
  inflating: butterflies/train/CAIRNS BIRDWING/102.jpg  
  inflating: butterflies/train/CAIRNS BIRDWING/080.jpg  
  inflating: butterflies/train/CAIRNS BIRDWING/081.jpg  
  inflating: butterflies/trai

[Full Code!](https://github.com/BrunoKrinski/PytorchClassification)

In [2]:
import os
import time
import argparse
import numpy as np
import torch.nn as nn
import torch, torchvision
import torch.optim as optim
import matplotlib.pyplot as plt

from tqdm import tqdm
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms

from torchvision.models import vgg16, VGG16_Weights
from torchvision.models import vgg16_bn, VGG16_BN_Weights
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models import maxvit_t, MaxVit_T_Weights

In [3]:
size = 224
image_transforms = { 
  'train': transforms.Compose([
      transforms.Resize(size=size),
      transforms.CenterCrop(size=size),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406],
                           [0.229, 0.224, 0.225])
  ]),
  'valid': transforms.Compose([
      transforms.Resize(size=size),
      transforms.CenterCrop(size=size),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406],
                           [0.229, 0.224, 0.225])
  ]),
  'test': transforms.Compose([
      transforms.Resize(size=size),
      transforms.CenterCrop(size=size),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406],
                           [0.229, 0.224, 0.225])
  ])
}

In [4]:
dataset = 'sports'
#dataset = 'butterflies'
train_directory = os.path.join(dataset, 'train')
valid_directory = os.path.join(dataset, 'valid')
test_directory = os.path.join(dataset, 'test')

In [5]:
bs = 32
num_classes = len(os.listdir(valid_directory))

print(num_classes)

100


In [6]:
data = {
  'train': datasets.ImageFolder(root=train_directory, transform=image_transforms['train']),
  'valid': datasets.ImageFolder(root=valid_directory, transform=image_transforms['valid']),
  'test': datasets.ImageFolder(root=test_directory, transform=image_transforms['test'])
}

In [7]:
idx_to_class = {v: k for k, v in data['train'].class_to_idx.items()}
print(idx_to_class)

{0: 'air hockey', 1: 'ampute football', 2: 'archery', 3: 'arm wrestling', 4: 'axe throwing', 5: 'balance beam', 6: 'barell racing', 7: 'baseball', 8: 'basketball', 9: 'baton twirling', 10: 'bike polo', 11: 'billiards', 12: 'bmx', 13: 'bobsled', 14: 'bowling', 15: 'boxing', 16: 'bull riding', 17: 'bungee jumping', 18: 'canoe slamon', 19: 'cheerleading', 20: 'chuckwagon racing', 21: 'cricket', 22: 'croquet', 23: 'curling', 24: 'disc golf', 25: 'fencing', 26: 'field hockey', 27: 'figure skating men', 28: 'figure skating pairs', 29: 'figure skating women', 30: 'fly fishing', 31: 'football', 32: 'formula 1 racing', 33: 'frisbee', 34: 'gaga', 35: 'giant slalom', 36: 'golf', 37: 'hammer throw', 38: 'hang gliding', 39: 'harness racing', 40: 'high jump', 41: 'hockey', 42: 'horse jumping', 43: 'horse racing', 44: 'horseshoe pitching', 45: 'hurdles', 46: 'hydroplane racing', 47: 'ice climbing', 48: 'ice yachting', 49: 'jai alai', 50: 'javelin', 51: 'jousting', 52: 'judo', 53: 'lacrosse', 54: 'log

In [8]:
train_data_size = len(data['train'])
valid_data_size = len(data['valid'])
test_data_size = len(data['test'])

print(train_data_size, valid_data_size, test_data_size)

13572 500 500


In [9]:
train_data_loader = DataLoader(data['train'], batch_size=bs, shuffle=True)
valid_data_loader = DataLoader(data['valid'], batch_size=bs, shuffle=True)
test_data_loader = DataLoader(data['test'], batch_size=bs, shuffle=True)

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "gpu")

In [11]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

[VGG16](https://miro.medium.com/v2/resize:fit:1400/1*NNifzsJ7tD2kAfBXt3AzEg.png)

[ResNet50](https://miro.medium.com/v2/resize:fit:1400/0*9LqUp7XyEx1QNc6A.png)

In [12]:
model = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
print(model)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 264MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [13]:
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
print(model)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 290MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [14]:
def get_model(name, num_classes):
    
  if name == 'vgg16':
    model = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)

    #for param in model.parameters():
    #    param.requires_grad = False

    fc_inputs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(fc_inputs, num_classes)

  elif name == 'vgg16_bn':
    model = vgg16_bn(weights=VGG16_BN_Weights.IMAGENET1K_V1)
    
    fc_inputs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(fc_inputs, num_classes)
        
  elif name == 'resnet50':
    model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
    
    fc_inputs = model.fc.in_features
    model.fc = nn.Linear(fc_inputs, num_classes)

  elif name == 'maxvit':
    model = maxvit_t(weights=MaxVit_T_Weights.DEFAULT)
    
    fc_inputs = model.classifier[5].in_features
    model.classifier[5] = nn.Linear(fc_inputs, num_classes)
    
  return model

In [15]:
model_name = 'vgg16'
#model_name = 'vgg16_bn'
#model_name = 'resnet50'
#model_name = 'maxvit'
model = get_model(model_name, num_classes=num_classes)
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [16]:
model = model.to(device)

In [17]:
loss_func = nn.CrossEntropyLoss()

In [18]:
lr = 0.00001
optimizer = optim.Adam(model.parameters(), lr=lr)

In [19]:
def train_and_validate(model, loss_criterion, optimizer, train_data_loader, valid_data_loader, train_data_size,
                       valid_data_size, device, output, epochs=25):      
  start = time.time()
  history = []
  best_loss = 100000.0
  best_epoch = None

  for epoch in range(epochs):
    epoch_start = time.time()
    print("Epoch: {}/{}".format(epoch+1, epochs))
        
    model.train()
        
    train_loss = 0.0
    train_acc = 0.0
    
    valid_loss = 0.0
    valid_acc = 0.0
        
    for i, (inputs, labels) in enumerate(tqdm(train_data_loader)):

      inputs = inputs.to(device)
      labels = labels.to(device)
            
      optimizer.zero_grad()
            
      outputs = model(inputs)
      loss = loss_criterion(outputs, labels)
      loss.backward()
            
      optimizer.step()
      train_loss += loss.item() * inputs.size(0)
            
      ret, predictions = torch.max(outputs.data, 1)
      correct_counts = predictions.eq(labels.data.view_as(predictions))
            
      acc = torch.mean(correct_counts.type(torch.FloatTensor))
            
      train_acc += acc.item() * inputs.size(0)
          
    with torch.no_grad():
      model.eval()
      for j, (inputs, labels) in enumerate(tqdm(valid_data_loader)):
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = loss_criterion(outputs, labels)
        valid_loss += loss.item() * inputs.size(0)

        ret, predictions = torch.max(outputs.data, 1)
        correct_counts = predictions.eq(labels.data.view_as(predictions))

        acc = torch.mean(correct_counts.type(torch.FloatTensor))

        valid_acc += acc.item() * inputs.size(0)
    
    if valid_loss < best_loss:
      best_loss = valid_loss
      best_epoch = epoch

    avg_train_loss = train_loss/train_data_size 
    avg_train_acc = train_acc/train_data_size

    avg_valid_loss = valid_loss/valid_data_size 
    avg_valid_acc = valid_acc/valid_data_size

    history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])        
    epoch_end = time.time()

    print("Epoch : {:03d}, Training: Loss - {:.4f}, Accuracy - {:.4f}%, Validation : Loss - {:.4f}, Accuracy - {:.4f}%, Time: {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))
    torch.save(model, f"{output}/model_{epoch}.pt")
            
  return model, history, best_epoch

In [20]:
output_path = 'expr'
!mkdir 'expr'

In [21]:
trained_model, history, best_epoch = train_and_validate(model, loss_func, optimizer, train_data_loader, valid_data_loader, 
                                                        train_data_size, valid_data_size, device, output_path, epochs=5)

Epoch: 1/5


100%|██████████| 16/16 [00:03<00:00,  4.41it/s]


Epoch : 000, Training: Loss - 2.9218, Accuracy - 30.8282%, Validation : Loss - 0.7508, Accuracy - 81.8000%, Time: 244.5796s
Epoch: 2/5


100%|██████████| 16/16 [00:03<00:00,  4.42it/s]


Epoch : 001, Training: Loss - 0.9744, Accuracy - 71.9422%, Validation : Loss - 0.3846, Accuracy - 89.2000%, Time: 239.2821s
Epoch: 3/5


...

Epoch : 002, Training: Loss - 0.5555, Accuracy - 83.1197%, Validation : Loss - 0.2771, Accuracy - 90.6000%, Time: 244.0031s
Epoch: 4/5


100%|██████████| 16/16 [00:03<00:00,  4.44it/s]


Epoch : 003, Training: Loss - 0.3506, Accuracy - 89.4268%, Validation : Loss - 0.2425, Accuracy - 91.6000%, Time: 241.2380s
Epoch: 5/5


100%|██████████| 16/16 [00:03<00:00,  4.43it/s]


Epoch : 004, Training: Loss - 0.2310, Accuracy - 92.8087%, Validation : Loss - 0.2074, Accuracy - 92.8000%, Time: 240.6328s


In [22]:
torch.save(history, f"{output_path}/{model_name}_history.pt")

In [31]:
def predict(model_path, test_image_name, idx_to_class, size = 224):

  image_transforms = { 
    'test': transforms.Compose([
      transforms.Resize(size=size),
      transforms.CenterCrop(size=size),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406],
                           [0.229, 0.224, 0.225])
    ])
  }
       
  transform = image_transforms['test']

  test_image = Image.open(test_image_name)
    
  test_image_tensor = transform(test_image)
  if torch.cuda.is_available():
    test_image_tensor = test_image_tensor.view(1, 3, 224, 224).cuda()
  else:
    test_image_tensor = test_image_tensor.view(1, 3, 224, 224)
    
  model = torch.load(model_path)

  with torch.no_grad():
    model.eval()

    out = model(test_image_tensor)
    ps = torch.exp(out)

    topk, topclass = ps.topk(3, dim=1)
    cls = idx_to_class[topclass.cpu().numpy()[0][0]]
    score = topk.cpu().numpy()[0][0]

    for i in range(3):
      print("Prediction", i+1, ":", idx_to_class[topclass.cpu().numpy()[0][i]], ", Score: ", topk.cpu().numpy()[0][i])

In [32]:
trained_model = 'expr/model_4.pt'
image_path = 'sports/test/archery/1.jpg'

predict(trained_model, image_path, idx_to_class, size)

Prediction 1 : archery , Score:  17257466.0
Prediction 2 : lacrosse , Score:  26.44792
Prediction 3 : jousting , Score:  24.696169
